# Bricklayer Robot

In [1]:
# Bookkeeping
# Python path for the local Drake and Manipulation
import sys

# Cool stuff
import shutil
import os as os
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource, GenerateHtml, GetDrakePath, PiecewisePolynomial)

/home/nikita/env/lib/python3.10/site-packages/mpld3/mplexporter/renderers/base.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [2]:
# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

Working in directory  /home/nikita/MIT_6_4212/project


In [3]:
# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

'/home/nikita/MIT_6_4212/project/../manipulation/manipulation/models/ground_model.sdf'

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [5]:
# System configs
# Playground: robot and basement
model_directives = """
directives:
- add_directives:
    file: package://manipulation/iiwa_and_wsg.dmd.yaml

- add_frame:
    name: ground_origin
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 0.0 ]}
      translation: [0.0, 0.0, 0.0]

- add_model:
    name: ground
    file: package://manipulation/ground_model.sdf

- add_weld:
    parent: ground_origin
    child: ground::ground_base

"""
# Generate many bricks under the scene
kNumberOfBricks = 100 # param

brick_string = """
- add_model:
    name: brick$
    file: package://manipulation/real_brick.sdf

"""
for i in range(0, kNumberOfBricks):
    brick_string_new = brick_string.replace("brick$", "brick_" + str(i))
    model_directives = model_directives + brick_string_new

# Brick source
X_WBrickSource = RigidTransform(np.array([0.5, 0.5, 0.05])) # param
X_WBrickTarget = RigidTransform(np.array([-0.3, 0.3, 0.35])) # param

In [6]:
# iiwa robot class
class IIWA_Painter():
    def __init__(self, traj_in=None):
        builder = DiagramBuilder()

        # set up the system of manipulation station
        self.station = MakeManipulationStation(model_directives=model_directives)
        builder.AddSystem(self.station)

        # set up plant
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj_in is not None:
            # traj and PseudoInverseController
            traj = traj_in[0]
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            # integrator and controller
            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa_position_measured"),
                self.controller.GetInputPort("iiwa_position"))

            # and trajectory source for the grip fingers as well
            finger_traj = traj_in[1]
            wsg_source = builder.AddSystem(TrajectorySource(finger_traj))
            wsg_source.set_name("wsg_command")
            builder.Connect(wsg_source.get_output_port(), self.station.GetInputPort("wsg_position"))

        # visualization
        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        # build and add diagram
        self.diagram = builder.Build()    
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        # resolve context
        context = self.CreateDefaultContext()
        self.diagram.Publish(context)

    # Helper to visualize frame
    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    # Helper to create default context
    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        # provide initial states
        q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context
    
    def reset_integrator(self, context, q0):
        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)
            
    def get_q0(self, contex):
        station_context = self.station.GetMyContextFromRoot(context)
        return self.station.GetOutputPort("iiwa_position_measured").Eval(station_context)
    
    # Helper to get current grip position
    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG
    
    # Lock bricks
    def lock_brick(self, context, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        brick_body.Lock(plant_context)
        
    # Unlock bricks
    def unlock_brick(self, context, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        brick_body.Unlock(plant_context)
    
    # Generate new brick
    def move_brick(self, context, X_WBrickSource, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        self.plant.SetFreeBodyPose(plant_context, brick_body, X_WBrickSource)
        self.visualize_frame("brick_source", X_WBrickSource)

    # Run simulation
    def paint(self, simulator, context, sim_duration=20.0):
        if context == None:
            context = self.CreateDefaultContext()
        
        if simulator == None:
            simulator = Simulator(self.diagram, context)
            simulator.set_target_realtime_rate(1.0)
            #simulator.get_mutable_integrator().set_target_accuracy(10)
            #simulator.get_mutable_integrator().set_maximum_step_size(100)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        
        return (simulator, context)

# PseudoInverseController class
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)


In [7]:
# Helper function
def merge_traj(traj_a, traj_b):
    traj = traj_a.copy()
    base_t = traj[-1][0] + 0.0001
    for trj in traj_b:
        traj.append([base_t + trj[0], trj[1], trj[2], trj[3]])
    return traj

def slow_down_traj(traj, k):
    for t in traj:
        t[0] = t[0] * k
    return traj

def slow_down_time(t, k):
    return [time * k for time in t]

In [8]:
# Instantiate the robot
painter = IIWA_Painter()

In [9]:
# Visualize the grip frame
painter.visualize_frame('gripper_current', painter.get_X_WG())

In [10]:
# 

In [11]:
# Setup trajectories
# Init pose
X_WG_init = painter.get_X_WG()

# Grip poses
X_BrickSourcePreG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),
                                   np.array([0, 0, 0.35]))
X_BrickSourceG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),
                                np.array([0, 0, 0.12]))

# Target poses
X_BrickTargetPreG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),# @ RotationMatrix.MakeYRotation(-np.pi/2),
                                   np.array([0, 0, 0.35]))
X_BrickTargetG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),# @ RotationMatrix.MakeYRotation(-np.pi/2),
                                np.array([0, 0, 0.12]))

# Finger poses
finger_opened = np.array([0.15]);
finger_closed = np.array([0.05]);

# Define trajectories: [time, pose, grip_pose, metainfo]
calibration_t = 2
traj_initial = [[0,             X_WG_init, finger_opened, 'initial'],
                [calibration_t, X_WBrickSource @ X_BrickSourcePreG, finger_opened, 'calibration']]

traj_grab_brick = [[0, X_WBrickSource @ X_BrickSourcePreG,    finger_opened, 'pre-grab'],
                   [2, X_WBrickSource @ X_BrickSourceG,    finger_opened, 'grab'],
                   [4, X_WBrickSource @ X_BrickSourceG,    finger_closed, 'grab, close']]

traj_move_to_place = [[0, X_WBrickSource @ X_BrickSourceG,    finger_closed, 'carry_begin'],
                      [6, X_WBrickTarget @ X_BrickTargetPreG, finger_closed, 'carry_end']]

traj_place_brick = [[0, X_WBrickTarget @ X_BrickTargetPreG, finger_closed, 'pre-put'],
                    [2, X_WBrickTarget @ X_BrickTargetG,    finger_closed, 'put'],
                    [4, X_WBrickTarget @ X_BrickTargetG,   finger_opened,  'put, open']]

traj_move_to_source = [[0, X_WBrickTarget @ X_BrickTargetG,    finger_opened, 'go_back_begin'],
                       [6, X_WBrickSource @ X_BrickSourcePreG, finger_opened, 'go_back_end/calibration']]

#
times = []
trajectories = traj_initial
for i in range(0, kNumberOfBricks):
    trajectories = merge_traj(trajectories, traj_grab_brick)
    trajectories = merge_traj(trajectories, traj_move_to_place)
    trajectories = merge_traj(trajectories, traj_place_brick)
    trajectories = merge_traj(trajectories, traj_move_to_source)
    times.append(trajectories[-1][0])

# Correct speed
slowdown_k = 0.6
trajectories = slow_down_traj(trajectories, slowdown_k)
times = slow_down_time(times, slowdown_k)
calibration_t = calibration_t * slowdown_k

traj = PiecewisePose.MakeLinear(np.array([t[0] for t in trajectories]),
                                np.array([t[1] for t in trajectories]))

traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [trajectories[0][0], trajectories[1][0]],
        np.hstack([[trajectories[0][2]], [trajectories[1][2]]]))
for i in range(2, len(trajectories)):
    traj_wsg_command.AppendFirstOrderSegment(trajectories[i][0], trajectories[i][2])

In [16]:
# Create robot
painter = IIWA_Painter([traj, traj_wsg_command])

# Create default context
context = painter.CreateDefaultContext()

def put_brick_in_warehouse(painter, brick_cnt):
    # Put all bricks in the warehouse
    w = 0.12
    v = 0.18
    h = 0.12
    brick_cnt = brick_cnt - 1
    for z in range (0, 5):
        for i in range (0, 5):
            for j in range (0, 5):
                X_WBrickWareHouse = RigidTransform(np.array([3.5 - w*i, 3.5 - v*j, 0.15 + h*z]))
                painter.move_brick(context, X_WBrickWareHouse, brick_cnt)
                painter.lock_brick(context, brick_cnt)
                if brick_cnt == 0:
                    return
                else:
                    brick_cnt = brick_cnt - 1
            
put_brick_in_warehouse(painter, kNumberOfBricks)

KeyboardInterrupt: 

In [ ]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = painter.paint(None, context, sim_duration=calibration_t)
q0 = painter.get_q0(ctx)
print("calibration q0= ", q0)

# Continue for all bricks
for i in range(0, kNumberOfBricks):
    if (i > 0):
        painter.lock_brick(context, i - 1)
    painter.move_brick(context, X_WBrickSource, i)
    painter.unlock_brick(context, i)
    sim, ctx = painter.paint(sim, ctx, sim_duration=times[i])
    painter.reset_integrator(ctx, q0)